In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
%matplotlib inline

In [ ]:
x = np.random.randn(50)
y = x**2

In [ ]:
for i in range(5):
    plt.scatter(x, y+i)

# Algerian Forest Fire Dataset

### To Predict->Tempearture

In [ ]:
df = pd.read_csv("Algerian_forest_fires_dataset_UPDATE.csv", header=1)
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.isna().sum()

In [ ]:
df[df.isnull().all(axis=1)]

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df.loc[:122, 'Region'] = 0#For Bejaia Region
df.loc[122:, 'Region'] = 1#For Sidi-Bel Abbes Region

In [ ]:
df.iloc[122]

In [ ]:
df.Region = df.Region.astype("int")

In [ ]:
df.dtypes

In [ ]:
#Checking Null Values
df.isna().sum()

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
df.shape

In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
df.iloc[120:125]

In [ ]:
#Drop 122nd index
df = df.drop(122).reset_index(drop=True)

In [ ]:
df.shape

### Formatting Column names

In [ ]:
df.columns

In [ ]:
df.columns = df.columns.str.strip()
df.columns

In [ ]:
#Changing to int type
df[['day', 'month', 'year', 'Temperature', 'RH', 'Ws']] = df[['day', 'month', 'year', 'Temperature', 'RH', 'Ws']].astype(int)

In [ ]:
df.dtypes

In [ ]:
#Changing other columns to float
objects = [feat for feat in df.columns if df[feat].dtypes=='object']
objects

In [ ]:
for feat in objects:
    if feat!='Classes':
        df[feat] = df[feat].astype(float)

In [ ]:
#Save to Cleaned dataset
df.to_csv("Algerian_forest_fires_cleaned_dataset.csv", index=False)

## Exploratory Data Analysis

In [ ]:
df_copy = df.drop(['day', 'month', 'year'], axis=1)
df_copy.head()

In [ ]:
df_copy.Classes.value_counts()

In [ ]:
#Making into only 2 classes
df_copy.Classes = np.where(df.Classes.str.contains("not fire"), 0, 1)
df_copy.Classes.value_counts()

In [ ]:
#Plotting data
plt.style.use('seaborn')
df_copy.hist(bins=50, figsize=(20, 15))
plt.show()

In [ ]:
#Percentage for pie-chart
percentage = df_copy.Classes.value_counts(normalize=True)*100
percentage

In [ ]:
classlabels = ["Fire", "Not Fire"]
plt.figure(figsize=(12, 7))
plt.pie(percentage, labels= classlabels, autopct='%1.1f%%')
plt.title("Pie Chart of Classes")
plt.show()

### Correlation

In [ ]:
df_copy.corr()

In [ ]:
sns.heatmap(df_copy.corr(), annot=True)

In [ ]:
sns.boxplot(df["FWI"])

In [ ]:
df.month.value_counts()

In [ ]:
df.Classes = np.where(df.Classes.str.contains("not fire"), "not fire", 'fire')

In [ ]:
#Monthly Fire Analysis
df_reg_1 = df_copy.loc[df["Region"]==1]
plt.subplots(figsize=(13, 6))
sns.set_style('whitegrid')
sns.countplot(x='month',hue='Classes', data=df[df.Region==1])
plt.ylabel("No. of Fires", weight='bold')
plt.xlabel("Months", weight='bold')
plt.title("Fire Analysis of Sidi-Bel Region", weight="bold")

In [ ]:
#Monthly Fire Analysis
df_reg_0 = df_copy.loc[df["Region"]==0]
plt.subplots(figsize=(13, 6))
sns.set_style('whitegrid')
sns.countplot(x='month',hue='Classes', data=df[df.Region==0])
plt.ylabel("No. of Fires", weight='bold')
plt.xlabel("Months", weight='bold')
plt.title("Fire Analysis of Bejia Region", weight="bold")
#plt.show()

# Machine Learning Models

### Lasso Regression

In [ ]:
df = pd.read_csv("Algerian_forest_fires_cleaned_dataset.csv")
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes

In [ ]:
df.drop(['day', 'month', 'year'], axis=1, inplace=True)

In [ ]:
df.Classes.value_counts()

In [ ]:
df.Classes = np.where(df.Classes.str.contains("not fire"), 0, 1)

In [ ]:
df.Classes.value_counts()

In [ ]:
# Independent and Dependent Features
X = df.drop('FWI', axis=1)
y = df['FWI']

In [ ]:
X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
type(X_train), type(X_test), type(y_train), type(y_test)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
#Feature Selection based on Correlation
X_train.corr()

In [ ]:
sns.heatmap(X_train.corr(), annot=True)

In [ ]:
for i in range(len(df.corr().columns)):
    print(i, df.corr().columns[i])

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        print("************************")
        print("i value = ",i,"col name = ",corr_matrix.columns[i] )
        for j in range(i):
            print("j value = ",j,"col name = ",corr_matrix.columns[j])
            if abs(corr_matrix.iloc[i, j])>threshold:
                print(i, corr_matrix.columns[i],j, corr_matrix.columns[j])
                col_name = corr_matrix.columns[i]
                col_corr.add(col_name)
            
    return col_corr

In [ ]:
correlation(X_train, 0.85)

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j])>threshold:
                col_name = corr_matrix.columns[i]
                col_corr.add(col_name)
    return col_corr

In [ ]:
corr_features = correlation(X_train, 0.85)
corr_features

In [ ]:
X_train.drop(corr_features, axis=1, inplace=True)
X_test.drop(corr_features, axis=1, inplace=True)
X_train.shape, X_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
type(X_train_scaled), type(X_test_scaled), type(X_train)

### Box Plot to understand Effect of Standard Scaler

In [ ]:
plt.subplots(figsize=(15, 5))

plt.subplot(1,2,1)
sns.boxplot(data=X_train)
plt.title("X_train before Scaling")

plt.subplot(1,2,2)
sns.boxplot(data=X_train_scaled)
plt.title("X_train after Scaling")

### Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)
y_pred = linreg.predict(X_test_scaled)
mae = mean_squared_error(y_test, y_pred)
score = r2_score(y_test, y_pred)

print("MSE = ",mae)
print("R2 Score = ",score)

plt.scatter(y_test, y_pred)

In [ ]:
residuals = y_test-y_pred
sns.histplot(residuals, kde=True)

In [ ]:
print("Coefficients : ",linreg.coef_)
print("Intercept : ", linreg.intercept_)
print("Number of features checked : ", linreg.n_features_in_)
print("Names of features checked : ",linreg.feature_names_in_)

# Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(X_train_scaled, y_train)
y_pred = lasso.predict(X_test_scaled)

mae = mean_squared_error(y_test, y_pred)
score = r2_score(y_test, y_pred)

print("MSE = ",mae)
print("R2 Score = ",score)

plt.scatter(y_test, y_pred)

In [ ]:
residuals = y_test-y_pred
sns.histplot(residuals, kde=True)

In [ ]:
print("Coefficients : ",lasso.coef_)
print("Intercept : ", lasso.intercept_)
print("Sparse Coefficient : ", lasso.sparse_coef_)
print("Number of features checked : ", lasso.n_features_in_)
print("Names of features checked : ",lasso.feature_names_in_)

# Cross Validation Lasso

In [ ]:
from sklearn.linear_model import LassoCV

In [ ]:
lassocv = LassoCV(cv=5)

In [ ]:
lassocv.fit(X_train_scaled, y_train)

In [ ]:
y_pred = lassocv.predict(X_test_scaled)
mae = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae, r2

In [ ]:
plt.scatter(y_test, y_pred)

### Ridge Regression

In [3]:
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(X_train_scaled, y_train)
y_pred = ridge.predict(X_test_scaled)

mae = mean_squared_error(y_test, y_pred)
score = r2_score(y_test, y_pred)

print("MSE = ",mae)
print("R2 Score = ",score)

plt.scatter(y_test, y_pred)

NameError: name 'X_train_scaled' is not defined

In [ ]:
from sklearn.linear_model import RidgeCV

ridgecv = RidgeCV(cv=5)
ridgecv.fit(X_train_scaled, y_train)
y_pred = ridgecv.predict(X_test_scaled)

mae = mean_squared_error(y_test, y_pred)
score = r2_score(y_test, y_pred)

print("MSE = ",mae)
print("R2 Score = ",score)

plt.scatter(y_test, y_pred)

In [ ]:
ridgecv.get_params()

### ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNetCV

elasticnetCV = ElasticNetCV(cv=5)
elasticnetCV.fit(X_train_scaled, y_train)
y_pred = elasticnetCV.predict(X_test_scaled)

mae = mean_squared_error(y_test, y_pred)
score = r2_score(y_test, y_pred)

print("MSE = ",mae)
print("R2 Score = ",score)

plt.scatter(y_test, y_pred)

In [ ]:
elasticnetCV.alphas_.shape

# Pickling Models

- ML Models
- Preprocessing Model StandardScaler

In [ ]:
scaler

In [ ]:
ridge

In [ ]:
import pickle

In [ ]:
pickle.dump(scaler, open("scaler.pkl", 'wb'))
pickle.dump(ridge, open("ridge.pkl", 'wb'))